Following http://modelai.gettysburg.edu/2013/cfr/cfr.pdf
#### Exercise 2.5: RPS Equilibrium (regret minimization).  hand-coding the algorithm

In [1]:
import random

In [36]:
# r, p, s = 0, 1, 2

training_iterations = 10000

strategy_profile_sum = [0,0,0]

player_one_regrets = [0,0,0]
player_two_regrets = [0,0,0]

def get_player_one_strategy():
    return get_strategy_from_regrets(player_one_regrets)
def get_player_two_strategy():
    return get_strategy_from_regrets(player_two_regrets)
def get_strategy_from_regrets(regrets):
    nonzero_regrets = [max(0,x) for x in regrets]
    if sum(nonzero_regrets) == 0:
        return [1/len(nonzero_regrets) for x in nonzero_regrets]
    return [x/sum(nonzero_regrets) for x in nonzero_regrets]
def sample_action(distribution):
    return random.choices(range(3), weights=distribution)[0]
def get_player_one_value(a1, a2):
    outcome = (a2-a1)%3
    if outcome == 0:
        return 0
    elif outcome == 1:
        return 1
    elif outcome == 2:
        return -1
def get_utilities(opponent_action):
    ans = []
    for i in [0,1,2]:
        ans.append(get_player_one_value(i,opponent_action))
    return ans
def accumulate(l1, l2):
    assert(len(l1) == len(l2))
    for i in range(len(l1)):
        l1[i] += l2[i]
    
for i in range(training_iterations):
    player_one_strat = get_player_one_strategy()
    player_two_strat = get_player_two_strategy()
    accumulate(strategy_profile_sum, player_one_strat)
    player_one_action = sample_action(player_one_strat)
    player_two_action = sample_action(player_two_strat)
    player_one_value = get_player_one_value(player_one_action, player_two_action)
    player_two_value = -player_one_value
    
    player_one_utilities = get_utilities(player_two_action)
    player_one_regret = [x-player_one_value for x in player_one_utilities]
    player_two_utilities = get_utilities(player_one_action)
    player_two_regret = [x-player_two_value for x in player_two_utilities]
    for j in range(len(player_one_regret)):
        player_one_regrets[j] += player_one_regret[j]
        player_two_regrets[j] += player_two_regret[j]
    

In [37]:
strategy_profile_sum

[3160.8931147910375, 3437.931922500671, 3401.1749627082904]

In [45]:
['{:.4}'.format(x/sum(strategy_profile_sum)) for x in strategy_profile_sum]

['0.3161', '0.3438', '0.3401']

Ok, it's hard to tell if that worked, since the equilbrium should be 1/3, 1/3, 1/3 and that could happen even if there was a bug.  Let's try this 538 riddler problem that I've always wanted to solve: https://fivethirtyeight.com/features/whats-the-optimal-way-to-play-horse/ (the riddler express problem) 
(by the way if I had done this while the contest was still running, this would be my submission: compute an equilibrium, then pick the highest-probability actions, then pick the best response to that would be.  submit that best response)

In [95]:
# again we'll just identify actions with their index integers: 0,1,2,3,4,5
score_probability = [0.7,0.9,0.8,0.8,1,.9]
num_of_actions = len(score_probability)

class Learner:
    def __init__(self, i_am_attacker):
        self.cumulative_regrets = [0 for i in range(num_of_actions)]
        self.strategy_profile_sum = [0 for i in range(num_of_actions)]
        self.i_am_attacker = i_am_attacker
    def sample_action(self):
        return random.choices(range(num_of_actions), weights=self._get_strategy())[0]
    def process_training_iteration(self, attacker_action, defender_action):
        strategy = self._get_strategy()
        for i in range(num_of_actions):
            self.strategy_profile_sum[i] += strategy[i]
        actual_payoff = self.get_my_payoff(attacker_action, defender_action)
        regrets = []
        for i in range(num_of_actions):
            if self.i_am_attacker:
                regret = self.get_my_payoff(i, defender_action)-actual_payoff
            else:
                regret = self.get_my_payoff(attacker_action, i)-actual_payoff
            self.cumulative_regrets[i] += regret
    def get_my_payoff(self, attacker_action, defender_action):
        multiplier = 1 if self.i_am_attacker else -1
        if attacker_action == defender_action:
            return 0
        # can either sample from score_probability here, or just use score_probability as the value.
        # I'm 96% sure we are allowed to do the latter and it should be better, so let's do it
        return multiplier * score_probability[attacker_action]
    def _get_strategy(self):
        nonzero_regrets = [max(0,x) for x in self.cumulative_regrets]
        if sum(nonzero_regrets) == 0:
            return [1/len(nonzero_regrets) for x in nonzero_regrets]
        return [x/sum(nonzero_regrets) for x in nonzero_regrets]

In [96]:
# train using regret minimization:

training_iterations = 100000

attacker = Learner(i_am_attacker=True)
defender = Learner(i_am_attacker=False)
for i in range(training_iterations):
    a_action = attacker.sample_action()
    d_action = defender.sample_action()
    
    attacker.process_training_iteration(a_action, d_action)
    defender.process_training_iteration(a_action, d_action)

In [97]:
# print results:
atk_strat = [x/sum(attacker.strategy_profile_sum) for x in attacker.strategy_profile_sum]
def_strat = [x/sum(defender.strategy_profile_sum) for x in defender.strategy_profile_sum]
print('atk probs:', ['{:.3}'.format(x) for x in atk_strat])
print('def probs:', ['{:.3}'.format(x) for x in def_strat])
total_payoff = 0
for i in range(num_of_actions):
    for j in range(num_of_actions):
        if i == j:
            continue
        total_payoff += atk_strat[i] * def_strat[j] * score_probability[i]
print('expected pts scored per kick:', total_payoff)

atk probs: ['0.186', '0.159', '0.176', '0.177', '0.142', '0.16']
def probs: ['1.67e-06', '0.224', '0.126', '0.125', '0.303', '0.222']
expected pts scored per kick: 0.6992083854389095


let's compare these values to the ones we get from a browser-based matrix-form nash solver: https://www.math.ucla.edu/~tom/gamesolve.html

browser solver:
```
The matrix is
 0 0.7 0.7 0.7 0.7 0.7
 0.9 0 0.9 0.9 0.9 0.9
 0.8 0.8 0 0.8 0.8 0.8
 0.8 0.8 0.8 0 0.8 0.8
 1 1 1 1 0 1
 0.9 0.9 0.9 0.9 0.9 0
The value is 0.69922.
An optimal strategy for Player I is:
 (0.19978,0.15538,0.17481,0.17481,0.13984,0.15538)
An optimal strategy for Player II is:
 (0.00111,0.22309,0.12597,0.12597,0.30078,0.22309)
```

regret minimization:
```
atk probs: ['0.182', '0.16', '0.176', '0.176', '0.145', '0.16']
def probs: ['2.62e-05', '0.222', '0.125', '0.123', '0.302', '0.227']
expected pts scored per kick: 0.6991933761145416
```

They all seem to be about the same down to 2 decimal points, so I think we're good

so if I used my aforementioned strategy, the most likely scoring probability would be on the 0.8 or 0.7 so I would defend a 0.8 or 0.7.
The most likely defending option would be the 1.0, so I would pick a 0.9 to score.

[how would I have done?](https://fivethirtyeight.com/features/can-you-construct-the-optimal-tournament/): The scoring is correct (the winners ended up aiming at the top 90%er), but the defending would be off: the best defenders defended the 1.0, not the 0.8 or the 0.7.